In [1]:
import time
notebook_start = time.time()

In [2]:
# Cell 1: Complete Environment Setup
# ========================================================

# 1. First, clean up everything - more thorough cleanup
import sys
import shutil
import os
import json

# Clean up problematic installations
!pip uninstall -y numpy torch torchvision torchaudio transformers peft bitsandbytes 2>/dev/null || echo "No packages to uninstall"

# Remove problematic directories manually
problematic_path = "/usr/local/lib/python3.11/dist-packages/~vidia-cudnn-cu12"
if os.path.exists(problematic_path):
    shutil.rmtree(problematic_path)

# Clear pip cache
!pip cache purge

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0
Files removed: 0


In [3]:
# 2. Install NumPy FIRST with clean environment
!pip install -q --ignore-installed numpy==1.26.4

# Force reload numpy if it was previously imported
if 'numpy' in sys.modules:
    del sys.modules['numpy']

# 3. Now import numpy FIRST before anything else
import numpy as np
print(f"NumPy version after clean install: {np.__version__}")

# 4. Install PyTorch with CUDA 12.1 (Kaggle's version)
!pip install -q torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121

# 5. Install transformer-related packages with compatible versions
!pip install -q transformers==4.41.2 peft==0.10.0 datasets==2.18.0 accelerate==0.29.1
!pip install -q bitsandbytes==0.43.0 einops==0.7.0

# 6. Handle gymnasium separately to avoid conflicts
!pip install -q gymnasium==0.29.0 --no-deps

# 7. Verify installations
import subprocess
import psutil
import torch
import torchvision

print("\n=== Core Package Versions ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")  # Should show 1.26.4
print(f"PyTorch: {torch.__version__}")
print(f"Torchvision: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n=== CUDA Information ===")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")

# 8. Now import transformer-related packages
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    pipeline
)
from typing import Optional
print("\n=== Transformer Packages Loaded Successfully ===")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.7.2 requires torch, which is not installed.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
torchmetrics 1.7.1 requires torch>=2.0.0, which is not installed.
pytorch-lightning 2.5.1.post0 requires torch>=2.1.0, which is not installed.
kaggle-environments 1.16.11 requires transformers>=4.33.1, which is not installed.
stable-baselines3 2.1.0 requires torch>=1.13, which is not installed.
sentence-transformers 3.4.1 requires torch>=1.11.0, which is not installed.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
fastai 2.

/tmp/ipykernel_69/2961444526.py:9: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  import numpy as np


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.8 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 12.2 MB/s eta 0:00:00:00:0100

/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


2025-05-29 22:09:34.201632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748556574.429529      69 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748556574.499302      69 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



=== Transformer Packages Loaded Successfully ===


In [4]:
# Cell 2: Model Loading
# =====================

# Define MODEL_NAME at the top of the cell (should match what is used in Cell 1)
MODEL_NAME = "gpt2"  # Change to "meta-llama/Llama-2-7b-chat-hf" for Llama

def print_memory():
    """Memory usage diagnostics for the environment"""
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU Memory: {gpu_mem:.2f}GB", end=" | ")
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

def load_model(model_name):
    """Load model with improved error handling and phi-1.5 specific settings"""
    print(f"\n=== Loading Model: {model_name} ===")
    print_memory()
    
    # Phi-1.5 specific configuration
    trust_remote_code = True  # Required for phi-1.5
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    
    # Quantization config for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch_dtype
    )
    
    try:
        print("Attempting quantized load...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            trust_remote_code=trust_remote_code,
            device_map="auto",
            torch_dtype=torch_dtype
        )
        
        print("\n✅ Model loaded successfully!")
        print_memory()
        return model
        
    except Exception as e:
        print(f"\n❌ Model loading failed: {str(e)}")
        print("Attempting standard load without quantization...")
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                device_map="auto" if torch.cuda.is_available() else None,
                torch_dtype=torch_dtype
            )
            print("\n✅ Model loaded successfully without quantization!")
            print_memory()
            return model
        except Exception as e:
            print(f"\n❌ Standard load failed: {str(e)}")
            print("Attempting CPU-only fallback...")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                device_map="cpu",
                torch_dtype=torch.float32
            )
            print("\n✅ Model loaded on CPU")
            print_memory()
            return model

model = load_model(MODEL_NAME)


=== Loading Model: gpt2 ===
RAM: 5.7% (1.3/31.4GB)
Attempting quantized load...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


❌ Model loading failed: No GPU found. A GPU is needed for quantization.
Attempting standard load without quantization...


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


✅ Model loaded successfully without quantization!
RAM: 7.5% (1.9/31.4GB)


In [5]:
# Cell 3: Tokenizer Setup
# =======================

def load_tokenizer(model_name):
    """Load and configure tokenizer"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("Tokenizer loaded successfully")
        return tokenizer
    except Exception as e:
        print(f"Tokenizer loading failed: {str(e)}")
        raise

tokenizer = load_tokenizer(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizer loaded successfully


In [6]:
# Cell 4: Robust Data Preparation
# =============================================

# 0. Set critical environment variables first
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"

# 1. Dataset preparation with multiple fallbacks
def prepare_dataset(file_path="/kaggle/input/database_0521.jsonl", max_samples=1000):
    """Prepare dataset with robust error handling"""
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"❌ Path not found: {file_path}")
            
        # Load JSONL file directly using datasets library
        dataset = load_dataset('json', data_files=file_path, split='train[:{}]'.format(max_samples))
        
        # If the dataset has multiple columns, we just want the text
        if 'text' not in dataset.features:
            # Try to find the first text-like column
            text_columns = [col for col in dataset.features if any(t in col.lower() for t in ['text', 'content', 'body'])]
            if text_columns:
                dataset = dataset.rename_column(text_columns[0], 'text')
            else:
                # If no text column found, combine all string columns
                string_columns = [col for col in dataset.features if dataset.features[col].dtype == 'string']
                if string_columns:
                    def combine_columns(examples):
                        return {'text': ' '.join(str(examples[col]) for col in string_columns)}
                    dataset = dataset.map(combine_columns)
                else:
                    raise ValueError("No text columns found in dataset")
        
        print(f"✅ Loaded dataset with {len(dataset)} samples")
        return dataset
        
    except Exception as e:
        print(f"\n❌ Dataset preparation failed: {str(e)}")
        print("Creating minimal fallback dataset...")
        return Dataset.from_dict({"text": ["Sample text " + str(i) for i in range(10)]})

# 2. Tokenization function
def safe_tokenize(examples):
    """Tokenization with explicit numpy workarounds"""
    try:
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=160,
            padding="max_length",
            return_tensors="pt"
        )
        # Convert to lists explicitly
        return {
            "input_ids": tokenized["input_ids"].tolist(),
            "attention_mask": tokenized["attention_mask"].tolist(),
            "labels": tokenized["input_ids"].tolist()
        }
    except RuntimeError as e:
        if "Numpy is not available" in str(e):
            # Fallback using pure Python
            return {
                "input_ids": [[0]*512],
                "attention_mask": [[1]*512],
                "labels": [[0]*512]
            }
        raise

try:
    print("\n=== Starting Processing ===")
    dataset = prepare_dataset()
    
    # Small batch test first
    test_batch = dataset.select(range(2))
    test_tokenized = test_batch.map(safe_tokenize, batched=True)
    
    # If test succeeds, process full dataset
    tokenized_dataset = dataset.map(safe_tokenize, batched=True, batch_size=4)
    tokenized_dataset.set_format(type='torch')
    
    print("✅ Processing completed successfully!")
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Creating minimal fallback dataset...")
    tokenized_dataset = Dataset.from_dict({
        "input_ids": [[0,1,2,3]],
        "attention_mask": [[1,1,1,1]],
        "labels": [[0,1,2,3]]
    })
    tokenized_dataset.set_format(type='torch')


=== Starting Processing ===

❌ Dataset preparation failed: ❌ Path not found: /kaggle/input/database_0521.jsonl
Creating minimal fallback dataset...


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Processing completed successfully!


In [7]:
# Cell 5: Training Configuration
# =============================

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# LoRA configuration
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,  
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],  # GPT-2 compatible modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    fan_in_fan_out=True
)

# Training arguments optimized for Kaggle
training_args = TrainingArguments(
    output_dir="/kaggle/working/phi1.5-lora-results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,  # Reduced for Kaggle
    learning_rate=2e-5,
    optim="adamw_torch",
    logging_steps=10,
    save_steps=500,
    fp16=torch.cuda.is_available(),
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    report_to="none"
)

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8606566809809635


In [8]:
# Cell 6: Training Execution
# =========================

def train_model(model, tokenized_dataset, training_args):
    """Execute the training process"""
    # Disable cache if gradient checkpointing is enabled
    if training_args.gradient_checkpointing:
        model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    
    print("Starting training...")
    print_memory()
    trainer.train()
    print("Training completed!")
    return trainer

trainer = train_model(model, tokenized_dataset, training_args)

Starting training...
RAM: 7.6% (1.9/31.4GB)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


Training completed!


In [9]:
# Cell 7: Enhanced Model Saving with Shard Support
# ===============================================

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[TrainingArguments] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification.
    Handles both single-file and sharded model formats.
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # For LoRA models - DON'T merge adapters before saving
    # We want to save the adapter separately
    print("💽 Saving model and adapter...")
    
    # Save the entire model (base model + adapter)
    model.save_pretrained(
        output_dir,
        safe_serialization=True,
        state_dict=model.state_dict()  # Save the complete state including LoRA
    )
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            elif hasattr(training_args, 'to_json_string'):
                with open(args_path, "w") as f:
                    f.write(training_args.to_json_string())
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify the adapter files were saved
    required_files = ['adapter_config.json', 'adapter_model.safetensors']
    missing_files = []
    for file in required_files:
        if not os.path.exists(os.path.join(output_dir, file)):
            missing_files.append(file)
    
    if missing_files:
        print(f"\n⚠️ Warning: Missing adapter files: {missing_files}")
        print("Trying alternative save method...")
        # Explicitly save the adapter
        model.save_pretrained(
            output_dir,
            safe_serialization=True,
            adapter_only=True  # This ensures adapter files are saved
        )
    
    print("\n🔍 Verifying saved files:")
    for file in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, file)) / 1024
        print(f"- {file} ({size:.2f} KB)")
    
    return output_dir

In [10]:
# Cell 8: Robust Model Loading and Testing with PEFT support
# ========================================================
def load_and_test_model(
    model_path: str = "/kaggle/working/gpt2-lora-trained", 
    max_length: int = 160,
    test_prompts: Optional[list] = None,
    is_peft_model: bool = True
):
    """
    Load and test a saved model with comprehensive error handling
    """
    print(f"\n🔍 Preparing to load model from: {model_path}")
    
    # Verify model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model directory {model_path} does not exist")
    
    # Show directory contents for debugging
    print("\n📂 Model directory contents:")
    for f in sorted(os.listdir(model_path)):
        size = os.path.getsize(os.path.join(model_path, f)) / 1024
        print(f"- {f} ({size:.2f} KB)")
    
    try:
        print("\n🔄 Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("\n🔄 Loading model...")
        if is_peft_model:
            # First check if we have adapter files
            adapter_files = [
                f for f in os.listdir(model_path) 
                if f.startswith('adapter_') or f == 'adapter_config.json'
            ]
            
            if not adapter_files:
                print("⚠️ No adapter files found. Loading as regular model.")
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
            else:
                print(f"Found adapter files: {adapter_files}")
                # Load base model first
                base_model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
                
                # Then load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path,
                    local_files_only=True
                )
                
                # Merge and unload for inference
                model = model.merge_and_unload()
        else:
            # For regular models
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                device_map="auto",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                local_files_only=True
            )
            
        print("\n🎉 Model loaded successfully!")
        
        # Default test prompts if none provided
        if test_prompts is None:
            test_prompts = [
                "What is hardware wallet?? ",
                "What is Proof of Work (PoW)?? ",
                "What is cryptography?? ",
                "What is Peer-to-Peer (P2P)?? ",
                "What is block chain?? ",
                "What is private key?? "
            ]
        
        # Create pipeline - REMOVED device parameter since we're using device_map="auto"
        print("\n🚀 Creating text generation pipeline...")
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        
        # Run tests
        print("\n🧪 Running generation tests...")
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\n🔹 Test {i}: {prompt}")
            output = pipe(
                prompt,
                max_length=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1,
                repetition_penalty=1.2
            )
            print("💬 Response:", output[0]['generated_text'])
            
        return model, tokenizer
        
    except Exception as e:
        print(f"\n❌ Critical error loading model: {str(e)}")
        print("\n🛠️ Debugging info:")
        print(f"- Path: {os.path.abspath(model_path)}")
        print(f"- Directory exists: {os.path.exists(model_path)}")
        if os.path.exists(model_path):
            print("- Contents:", os.listdir(model_path))
        raise

In [11]:
# Main execution
if __name__ == "__main__":
    model_path = "/kaggle/working/gpt2-lora-trained"
    
    # Save model artifacts
    save_model_artifacts(model, tokenizer, training_args)
    
    # Load with explicit path and PEFT flag
    load_and_test_model(model_path, is_peft_model=True)
    
    # Test with custom prompts
    custom_prompts = [
        "What is software wallet, and what's the difference between hardware and software wallet? ",
        "What is PoW? ",
        "Explain PoW in 1 sentence. ",
        "Describe the key features of PoW using 3 words. ",
        "What is PoM? Is it something related to cryptography? ",
        "What is a cryptographic product? ",
        "What is P2P? ",
        "What is block chain? ",
        "What is public key, and what's the difference between private and public key? "
    ]
    load_and_test_model(model_path, test_prompts=custom_prompts, is_peft_model=True)


💾 Saving model artifacts to: /kaggle/working/gpt2-lora-trained
💽 Saving model and adapter...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔤 Saving tokenizer...
📝 Saving training arguments...

🔍 Verifying saved files:
- special_tokens_map.json (0.13 KB)
- training_args.json (3.86 KB)
- README.md (4.96 KB)
- tokenizer_config.json (0.49 KB)
- tokenizer.json (2058.55 KB)
- merges.txt (445.62 KB)
- adapter_config.json (0.66 KB)
- adapter_model.safetensors (9227.88 KB)
- vocab.json (779.45 KB)

🔍 Preparing to load model from: /kaggle/working/gpt2-lora-trained

📂 Model directory contents:
- README.md (4.96 KB)
- adapter_config.json (0.66 KB)
- adapter_model.safetensors (9227.88 KB)
- merges.txt (445.62 KB)
- special_tokens_map.json (0.13 KB)
- tokenizer.json (2058.55 KB)
- tokenizer_config.json (0.49 KB)
- training_args.json (3.86 KB)
- vocab.json (779.45 KB)

🔄 Loading tokenizer...

🔄 Loading model...
Found adapter files: ['adapter_config.json', 'adapter_model.safetensors']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎉 Model loaded successfully!

🚀 Creating text generation pipeline...

🧪 Running generation tests...

🔹 Test 1: What is hardware wallet?? 
💬 Response: What is hardware wallet??  It's a simple, fast and secure way to store your money in an easy-touse digital currency that you can use for transactions anywhere on the internet. It doesn't have any central bank controls or restrictions like Bitcoin does (like it did with Visa). The only difference between this technology and traditional paper wallets are their security features:
The main advantage of using cash over electronic currencies such as dollars isn´t just convenience but also because there aren` t many other ways around them - some even require having physical keys too! There will always be someone waiting at home who may need access codes from within Bitcoins which could easily change hands if needed; all these things should give users confidence when dealing directly into online banking networks without breaking banks' trust sys

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



🎉 Model loaded successfully!

🚀 Creating text generation pipeline...

🧪 Running generation tests...

🔹 Test 1: What is software wallet, and what's the difference between hardware and software wallet? 
💬 Response: What is software wallet, and what's the difference between hardware and software wallet?  In this article I will show you how to use a simple Bitcoin Wallet (BWP) with BWP-based wallets.
The goal of my blog post was not just for beginners who are newbies in blockchain technology but also those using digital currencies like bitcoin or litecoin that do business as btc/lose money without having any actual account details so all information about your coins can be easily retrieved from an open source site such BitFury. This means if anyone wants more than one type of coin they must buy them through their own website before purchasing it on here: http://bitfurencewalletchainweb.com/. The easiest way would have been selling bitcoins directly into MtGox

🔹 Test 2: What is PoW? 
💬 Re

In [12]:

notebook_end = time.time()
print(f"Total notebook execution time: {notebook_end - notebook_start:.2f} seconds")

Total notebook execution time: 335.95 seconds
